## Import Library

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import os
import cv2

In [ ]:
import keras
import keras_vggface
import keras_vggface.utils
from keras_vggface.vggface import VGGFace
from keras import Model
from keras import layers
from keras.layers import Flatten, Input, Dropout
from keras.layers.core import Dense
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import mtcnn
from mtcnn.mtcnn import MTCNN
from keras_vggface.utils import preprocess_input
from scipy.spatial.distance import cosine

## Load Pre-trained Model

In [ ]:
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

In [ ]:
model.trainable = False

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.2),
])

In [ ]:
# Choose `avg_pool` as the last layer of your base model
last_layer = model.get_layer('avg_pool')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
inputs = Input(shape=(224, 224, 3))

x = data_augmentation(inputs)
x = model(x)
x = Flatten(name='flatten')(x)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(1024, activation='relu', name='fc3')(x) 
x = Dense(1024, activation='relu', name='fc4')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu', name='fc5')(x)
x = Dense(512, activation='relu', name='fc6')(x) 
x = Dense(512, activation='relu', name='fc7')(x) 
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', name='fc8')(x)
x = Dense(256, activation='relu', name='fc9')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu', name='fc10')(x)
out = Dense(105, name='classifier')(x)

In [ ]:
custom_model = Model(inputs, out)

## Prepare Dataset

In [ ]:
# Define our example directories and files
base_dir = 'C:\\Users\\ACER\\Capstone\\dataset\\image_sorted'

train_dir = os.path.join( base_dir, 'train')
test_dir = os.path.join( base_dir, 'test')

In [ ]:
train_dataset = keras.utils.image_dataset_from_directory(train_dir,
                                                         shuffle=True,
                                                         batch_size=8,
                                                         image_size=(224, 224))

test_dataset = keras.utils.image_dataset_from_directory(test_dir,
                                                         shuffle=True,
                                                         batch_size=8,
                                                         image_size=(224, 224))

## Set Optimizer and Loss Functions

In [ ]:
learning_rate = 0.0001
custom_model.compile(optimizer=RMSprop(learning_rate=learning_rate), 
                     loss=SparseCategoricalCrossentropy(from_logits=True), 
                     metrics=['accuracy'])

## Set Callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        '''
        Halts the training after reaching 60 percent accuracy
        
        Args:
        epoch (integer) - index of epoch (required but unused in the function definition below)
        logs (dict) - metric results from the training epoch
        '''
        # Check accuracy
        if(logs.get('loss') < 0.7):
            # Stop if threshold is met
            print("\nLoss is lower than 0.7 so cancelling training!")
            self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

## Train Model

In [ ]:
history = custom_model.fit(train_dataset, 
                           validation_data=test_dataset, 
                           steps_per_epoch = len(train_dataset),
                           epochs = 50,
                           validation_steps = len(test_dataset),
                           verbose = 2, 
                           callbacks=[callbacks])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

## Extractor

In [ ]:
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = plt.imread(filename)
    
    # create the detector, using default weights
    detector = MTCNN()
    
    # detect faces in the image
    results = detector.detect_faces(pixels)
    
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

## Verification Check

### With Fine-Tuning

In [ ]:
filenames = ["C:\\Users\\ACER\\Capstone\\verification\\olla1.jpg" , 
             "C:\\Users\\ACER\\Capstone\\verification\\gita2.jpg"]

faces = [extract_face(f) for f in filenames]
samples = np.asarray(faces, "float32")
samples = preprocess_input(samples, version=2)

# perform prediction
embeddings = custom_model.predict(samples)
thresh = 0.0015
score = cosine(embeddings[0], embeddings[1])
if score <= thresh:
    print( " >face is a match (%.4f <= %.4f) " % (score, thresh))
else:
    print(" >face is NOT a match (%.4f > %.4f)" % (score, thresh))

### Without Fine-Tuning

In [ ]:
filenames = ["C:\\Users\\ACER\\Capstone\\verification3\\irene1.jpg" , 
             "C:\\Users\\ACER\\Capstone\\verification3\\irene2.jpg"]

faces = [extract_face(f) for f in filenames]
samples = np.asarray(faces, "float32")
samples = preprocess_input(samples, version=2)
model = VGGFace(model= "resnet50" , include_top=False, input_shape=(224, 224, 3), pooling= "avg" )

# perform prediction
embeddings = model.predict(samples)
thresh = 0.5
score = cosine(embeddings[0], embeddings[1])
if score <= thresh:
    print( " >face is a match (%.3f <= %.3f) " % (score, thresh))
else:
    print(" >face is NOT a match (%.3f > %.3f)" % (score, thresh))

## Save the Model

In [ ]:
custom_model.save('model/model-6-8079.h5')

## Convert Model to Tflite

In [ ]:
vggface_resnet_converter = tf.lite.TFLiteConverter.from_keras_model(custom_model)
vggface_resnet_converter.optimizations = [tf.lite.Optimize.DEFAULT]
vggface_resnet_tflite = vggface_resnet_converter.convert()

### Save the Tflite File

In [ ]:
with open('model/model-6.tflite', 'wb') as f:
    f.write(vggface_resnet_tflite)